# Building a Structured Data Cleaning Pipeline for Retail Transactions
## Why Data Cleaning Matters

In real-world scenarios, datasets are rarely analysis-ready.<br>
Inconsistent formats, invalid entries, missing values, and logical errors 
can severely impact analytical results and model performance.

Before any modeling or visualization<br>
a structured data cleaning process is essential.

This notebook demonstrates how to transform a messy retail transaction dataset 
into a reliable and analysis-ready dataset using a systematic pipeline approach.

## Data Cleaning Strategy

The cleaning process follows these stages:

1. Structural Cleaning (duplicates, formatting issues)
2. Numeric Standardization (currency symbols, type conversion)
3. Missing Value Handling
4. Business Rule Validation
5. Text Standardization (phone, email)
6. Date Normalization
7. Final Data Quality Assessment

In [1]:
#import libraries
import pandas as pd
import re

In [2]:
#import data
#This is not real data. It is just for practice :)

df=pd.read_excel("unclean_data.xlsx")
df.head()

,order_id,customer_name,phone,email,product,category,quantity,unit_price,total_price,purchase_date,payment_method,city,discount_code
0,1001,John Smith,+1 202-555-0143,john.smith@gmail.com,Laptop,Electronics,1,1200,1200,2023-05-12 00:00:00,Credit Card,New York,NaN
1,1002,Emily Johnson,2025550187,emily_johnson[at]gmail.com,Headphones,Electronics,2,85,NaN,2023-12-05 00:00:00,card,New York,OFF10
2,1003,Michael Brown,NaN,m.brown@gmail,Smartphone,Electronics,1,950,950$,2023-13-01,Cash,Chicago,NaN
3,1004,Sophia Davis,+44 7700 900123,sophia.davis@gmail.com,T-shirt,Clothing,3,25$,75,2023-05-15 00:00:00,Online Payment,London,SUMMER
4,1005,Daniel Wilson,+1 202-555-0111,daniel.wilson@yahoo.com,Shoes,Clothing,2,120,240,15/05/2023,cash,Toronto,NaN


*In the first step, we need to understand the dataset so that we can identify the problematic points.*
**Understand first. Clean later.**

In [3]:
df.shape

(22, 13)

In [4]:
#To find out how much Nun data we have in each column
df.isnull().sum()

order_id           0
customer_name      0
phone              1
email              1
product            0
category           0
quantity           0
unit_price         1
total_price        1
purchase_date      0
payment_method     1
city               1
discount_code     16
dtype: int64

In [5]:
#Non-null count and data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        22 non-null     int64 
 1   customer_name   22 non-null     object
 2   phone           21 non-null     object
 3   email           21 non-null     object
 4   product         22 non-null     object
 5   category        22 non-null     object
 6   quantity        22 non-null     int64 
 7   unit_price      21 non-null     object
 8   total_price     21 non-null     object
 9   purchase_date   22 non-null     object
 10  payment_method  21 non-null     object
 11  city            21 non-null     object
 12  discount_code   6 non-null      object
dtypes: int64(2), object(11)
memory usage: 2.4+ KB


In [6]:
#Checking for duplicate records
df["order_id"].duplicated().sum()

2

## Handling Duplicate Records

Duplicate transactions can distort revenue and customer analysis.
We remove duplicate records based on order_id to ensure transaction-level uniqueness.

In [7]:
print("Before removing duplicates:", df.shape)

df = df.drop_duplicates(subset="order_id")

print("After removing duplicates:", df.shape)

Before removing duplicates: (22, 13)
After removing duplicates: (20, 13)


## Cleaning Numeric Columns

The unit_price and total_price columns contain currency symbols and formatting issues.
We need to:
- Remove currency symbols ($)
- Remove commas
- Convert values to numeric
- Handle invalid entries

In [8]:
# convert to string 
df['unit_price']=df['unit_price'].astype(str)
df['total_price']=df['total_price'].astype(str)

# Remove $ and commas
df["unit_price"] = (
    df["unit_price"]
    .str.replace("$", "", regex=False)
    .str.replace(",", "", regex=False)
)

df["total_price"] = (
    df["total_price"]
    .str.replace("$", "", regex=False)
    .str.replace(",", "", regex=False)
)

# Convert to numeric
df["unit_price"] = pd.to_numeric(df["unit_price"], errors="coerce")
df["total_price"] = pd.to_numeric(df["total_price"], errors="coerce")

df.head()

,order_id,customer_name,phone,email,product,category,quantity,unit_price,total_price,purchase_date,payment_method,city,discount_code
0,1001,John Smith,+1 202-555-0143,john.smith@gmail.com,Laptop,Electronics,1,1200.0,1200.0,2023-05-12 00:00:00,Credit Card,New York,NaN
1,1002,Emily Johnson,2025550187,emily_johnson[at]gmail.com,Headphones,Electronics,2,85.0,NaN,2023-12-05 00:00:00,card,New York,OFF10
2,1003,Michael Brown,NaN,m.brown@gmail,Smartphone,Electronics,1,950.0,950.0,2023-13-01,Cash,Chicago,NaN
3,1004,Sophia Davis,+44 7700 900123,sophia.davis@gmail.com,T-shirt,Clothing,3,25.0,75.0,2023-05-15 00:00:00,Online Payment,London,SUMMER
4,1005,Daniel Wilson,+1 202-555-0111,daniel.wilson@yahoo.com,Shoes,Clothing,2,120.0,240.0,15/05/2023,cash,Toronto,NaN


## 4. Handling Missing Values

Before applying business logic validation, we need to address missing values
to ensure consistency and avoid unintended errors.

Missing handling depends on business context.

We distinguish between critical and non-critical missing values.
Critical fields (quantity, unit_price) require removal,
while non-critical fields are filled with placeholders.

In [9]:
df = df[df["quantity"].notna()]
df = df[df["unit_price"].notna()]
df.shape

(18, 13)

In [10]:
df["payment_method"] = df["payment_method"].fillna("unknown")
df["city"] = df["city"].fillna("unknown")
df.head()

,order_id,customer_name,phone,email,product,category,quantity,unit_price,total_price,purchase_date,payment_method,city,discount_code
0,1001,John Smith,+1 202-555-0143,john.smith@gmail.com,Laptop,Electronics,1,1200.0,1200.0,2023-05-12 00:00:00,Credit Card,New York,NaN
1,1002,Emily Johnson,2025550187,emily_johnson[at]gmail.com,Headphones,Electronics,2,85.0,NaN,2023-12-05 00:00:00,card,New York,OFF10
2,1003,Michael Brown,NaN,m.brown@gmail,Smartphone,Electronics,1,950.0,950.0,2023-13-01,Cash,Chicago,NaN
3,1004,Sophia Davis,+44 7700 900123,sophia.davis@gmail.com,T-shirt,Clothing,3,25.0,75.0,2023-05-15 00:00:00,Online Payment,London,SUMMER
4,1005,Daniel Wilson,+1 202-555-0111,daniel.wilson@yahoo.com,Shoes,Clothing,2,120.0,240.0,15/05/2023,cash,Toronto,NaN


## Problems with the phone
Our goal:\
We want to:\
Keep only numbers\
All numbers have a single format\
Declare invalid cases as invalid\
Check for logical length

In [11]:
df["phone"] = df["phone"].astype(str)
df["phone"] = df["phone"].str.replace(r"\D", "", regex=True)
df.loc[df["phone"].str.len() < 9, "phone"] = "invalid"

df.head()

,order_id,customer_name,phone,email,product,category,quantity,unit_price,total_price,purchase_date,payment_method,city,discount_code
0,1001,John Smith,12025550143,john.smith@gmail.com,Laptop,Electronics,1,1200.0,1200.0,2023-05-12 00:00:00,Credit Card,New York,NaN
1,1002,Emily Johnson,2025550187,emily_johnson[at]gmail.com,Headphones,Electronics,2,85.0,NaN,2023-12-05 00:00:00,card,New York,OFF10
2,1003,Michael Brown,invalid,m.brown@gmail,Smartphone,Electronics,1,950.0,950.0,2023-13-01,Cash,Chicago,NaN
3,1004,Sophia Davis,447700900123,sophia.davis@gmail.com,T-shirt,Clothing,3,25.0,75.0,2023-05-15 00:00:00,Online Payment,London,SUMMER
4,1005,Daniel Wilson,12025550111,daniel.wilson@yahoo.com,Shoes,Clothing,2,120.0,240.0,15/05/2023,cash,Toronto,NaN


## Problems with email
Wrong format\
Incomplete value\
Empty line\
Duplicate email

In [12]:
#Convert to string and remove extra spaces
df["email"] = df["email"].astype(str).str.strip()

#Fixing some common errors
df["email"] = df["email"].str.replace("[at]", "@", regex=False)

# Validation with Regex
email_pattern = r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$'
df["email"] = df["email"].apply(
    lambda x: x if re.match(email_pattern, x) else "invalid"
)

df.head()

,order_id,customer_name,phone,email,product,category,quantity,unit_price,total_price,purchase_date,payment_method,city,discount_code
0,1001,John Smith,12025550143,john.smith@gmail.com,Laptop,Electronics,1,1200.0,1200.0,2023-05-12 00:00:00,Credit Card,New York,NaN
1,1002,Emily Johnson,2025550187,emily_johnson@gmail.com,Headphones,Electronics,2,85.0,NaN,2023-12-05 00:00:00,card,New York,OFF10
2,1003,Michael Brown,invalid,invalid,Smartphone,Electronics,1,950.0,950.0,2023-13-01,Cash,Chicago,NaN
3,1004,Sophia Davis,447700900123,sophia.davis@gmail.com,T-shirt,Clothing,3,25.0,75.0,2023-05-15 00:00:00,Online Payment,London,SUMMER
4,1005,Daniel Wilson,12025550111,daniel.wilson@yahoo.com,Shoes,Clothing,2,120.0,240.0,15/05/2023,cash,Toronto,NaN


## Problems with the purchase_date column
Different formats

2023-05-12 00:00:00

15/05/2023


Invalid date

2023-13-01 ❌ (month 13 does not exist)

2023-15-01 ❌

In [13]:
df["purchase_date"] = pd.to_datetime(
    df["purchase_date"],
    errors="coerce"
).dt.strftime("%Y-%m-%d")

df.loc[df["purchase_date"].isna(), "purchase_date"] = "invalid"

df.head()

,order_id,customer_name,phone,email,product,category,quantity,unit_price,total_price,purchase_date,payment_method,city,discount_code
0,1001,John Smith,12025550143,john.smith@gmail.com,Laptop,Electronics,1,1200.0,1200.0,2023-05-12,Credit Card,New York,NaN
1,1002,Emily Johnson,2025550187,emily_johnson@gmail.com,Headphones,Electronics,2,85.0,NaN,2023-12-05,card,New York,OFF10
2,1003,Michael Brown,invalid,invalid,Smartphone,Electronics,1,950.0,950.0,invalid,Cash,Chicago,NaN
3,1004,Sophia Davis,447700900123,sophia.davis@gmail.com,T-shirt,Clothing,3,25.0,75.0,2023-05-15,Online Payment,London,SUMMER
4,1005,Daniel Wilson,12025550111,daniel.wilson@yahoo.com,Shoes,Clothing,2,120.0,240.0,2023-05-15,cash,Toronto,NaN


## 5. Business Rule Validation

After structural cleaning, we validate logical consistency 
based on simple retail business rules.

In [14]:
#rule 1: Quantity Must Be Positive
print("Before quantity validation:", df.shape)

df = df[df["quantity"] > 0]

print("After quantity validation:", df.shape)

Before quantity validation: (18, 13)
After quantity validation: (16, 13)


In [15]:
#rule2: Unit Price Must Be Positive
print("Before unit price validation:", df.shape)

df = df[df["unit_price"] > 0]

print("After unit price validation:", df.shape)

Before unit price validation: (16, 13)
After unit price validation: (16, 13)


In [16]:
# total_price must be equal to: quantity * unit_price
#The data type is decimal, so we round the number to 2 decimal places.

df["calculated_total"] = df["quantity"] * df["unit_price"].round(2)

# Find inconsistent rows
inconsistent = df[df["total_price"].round(2) != df["calculated_total"]]

#Modify total_price
df["total_price"] = df["calculated_total"]

#Remove auxiliary column
df = df.drop(columns=["calculated_total"])

In [17]:
#save clean data in new file

df.to_excel("clean_data.xlsx")

## Final Reflection

Data cleaning is not only about formatting corrections.<br>
It enforces logical consistency, structural integrity,<br>
and business-level validation.

A well-designed cleaning pipeline significantly improves<br>
analytical reliability.